<a href="https://colab.research.google.com/github/alexandresvifpb/assert_iaiot/blob/main/project_softex_ia_XGBClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

# Carregar os dados
url = 'https://raw.githubusercontent.com/alexandresvifpb/assert_iaiot/main/letter-recognition.data'
letter_recognition = pd.read_csv(url, header=None, names=['lettr', 'x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx'])
# print(letter_recognition)

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 2.1. Verificar e remover valores nulos
letter_recognition = letter_recognition.dropna()
# print("numero de linhas sem valores nulos: ",len(letter_recognition))

# 2.2. Converter labels para números
label_encoder = LabelEncoder()
letter_recognition['lettr'] = label_encoder.fit_transform(letter_recognition['lettr'])

# 2.3. Verificar se existe correspondência entre features
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = letter_recognition.corr()

# Criar um mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlação Letter Recognition")
plt.show()

# 2.4. Normalizar as features na escala 0..1
scaler = MinMaxScaler()
letter_recognition_scaled = pd.DataFrame(scaler.fit_transform(letter_recognition.drop(columns=['lettr'])), columns=letter_recognition.columns[1:])

# Concatenar os labels (lettr) com as features normalizadas
letter_recognition_processed = pd.concat([letter_recognition['lettr'], letter_recognition_scaled], axis=1)
# print("\nDataset após pré-processamento:\n", letter_recognition_processed.head())

In [ ]:
# 3.1 Gráfico de barras por classes
# Contar a quantidade de instâncias por classe
class_counts = letter_recognition['lettr'].value_counts()

# Criar um gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(x=class_counts.index, y=class_counts.values, palette='viridis')
plt.title('Quantidade de Instâncias por Classe')
plt.xlabel('Classe')
plt.ylabel('Quantidade')
plt.show()

# 3.2. Gráfico de dispersão com linha de regressão
plt.figure(figsize=(12, 6))
sns.regplot(x='x-box', y='lettr', data=letter_recognition, scatter_kws={'s': 20, 'alpha': 0.5}, line_kws={'color': 'red'})
plt.title('Gráfico de Dispersão com Linha de Regressão')
plt.xlabel('x-box')
plt.ylabel('lettr')
plt.show()

# 3.3. Gráfico de distribuição (histograma)
plt.figure(figsize=(10, 6))
sns.histplot(letter_recognition['x-box'], bins=30, color='skyblue', kde=True)
plt.title('Histograma da Feature x-box')
plt.xlabel('x-box')
plt.ylabel('Frequência')
plt.show()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import numpy as np

# 4.1 Separar os dados de treinamento e teste (80% e 20%)
X = letter_recognition_processed.drop(columns=['lettr'])
y = letter_recognition_processed['lettr']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4.2 Treinar os modelos XGBoost
# Modelo XGBoost 1
xgb_model1 = xgb.XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y)), random_state=42)
xgb_model1.fit(X_train, y_train)

# Modelo XGBoost 2
xgb_model2 = xgb.XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y)), max_depth=10, learning_rate=0.05, random_state=42)
xgb_model2.fit(X_train, y_train)

# 4.3 Executar 10 vezes o Train-Test-Split
num_executions = 2
accuracies_xgb_model1 = cross_val_score(xgb_model1, X, y, cv=num_executions, scoring='accuracy')
accuracies_xgb_model2 = cross_val_score(xgb_model2, X, y, cv=num_executions, scoring='accuracy')

# 4.4 Apresentar a média das 10 execuções dos resultados de acurácia de todos os modelos
mean_accuracy_xgb_model1 = accuracies_xgb_model1.mean()
mean_accuracy_xgb_model2 = accuracies_xgb_model2.mean()

print(f'Média de Acurácia para XGBoost Modelo 1: {mean_accuracy_xgb_model1:.3%}')
print(f'Média de Acurácia para XGBoost Modelo 2: {mean_accuracy_xgb_model2:.3%}')

Média de Acurácia para XGBoost Modelo 1: 94.590%
Média de Acurácia para XGBoost Modelo 2: 92.845%
